# Classification by Wine Type, Part 2

## Wine Data
Data from http://archive.ics.uci.edu/ml/datasets/Wine+Quality

### Citations
<pre>
Dua, D. and Karra Taniskidou, E. (2017). 
UCI Machine Learning Repository [http://archive.ics.uci.edu/ml/index.php]. 
Irvine, CA: University of California, School of Information and Computer Science.
</pre>

<pre>
P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
Modeling wine preferences by data mining from physicochemical properties.
In Decision Support Systems, Elsevier, 47(4):547-553. ISSN: 0167-9236.
</pre>

Available at:
- [@Elsevier](http://dx.doi.org/10.1016/j.dss.2009.05.016)
- [Pre-press (pdf)](http://www3.dsi.uminho.pt/pcortez/winequality09.pdf)
- [bib](http://www3.dsi.uminho.pt/pcortez/dss09.bib)

## Setup

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

red_wine = pd.read_csv('data/winequality-red.csv')
white_wine = pd.read_csv('data/winequality-white.csv', sep=';')
wine = pd.concat([
    white_wine.assign(kind='white'), red_wine.assign(kind='red')
])

Since we completed our EDA in the [`wine.ipynb`](../ch_09/wine.ipynb) notebook for last chapter, we will just look at some rows to refresh our memory of the data rather than repeating the EDA here.

In [ ]:
wine.sample(5, random_state=10)

## Train Test Split
As in chapter 9, we will try to predict if a wine is red or white based on its chemical properties:

In [ ]:
from sklearn.model_selection import train_test_split

wine_y = np.where(wine.kind == 'red', 1, 0)
wine_X = wine.drop(columns=['quality', 'kind'])

w_X_train, w_X_test, w_y_train, w_y_test = train_test_split(
    wine_X, wine_y, test_size=0.25, random_state=0, stratify=wine_y
)

## Logistic Regression Classification of Red and White Wines from Chapter 9
This was the result from chapter 9 for reference:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

white_or_red = Pipeline([
    ('scale', StandardScaler()), 
    ('lr', LogisticRegression(random_state=0))
]).fit(w_X_train, w_y_train)

kind_preds = white_or_red.predict(w_X_test)

The model performed very well without any tuning:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(w_y_test, kind_preds))

In [ ]:
from ml_utils.classification import plot_roc

plot_roc(w_y_test, white_or_red.predict_proba(w_X_test)[:,1])

In [ ]:
from ml_utils.classification import confusion_matrix_visual

confusion_matrix_visual(w_y_test, kind_preds, ['white', 'red'])

## Dimensionality Reduction
### Variance Threshold
Features with little to no variance don't contribute much to our classification model. We can remove those and work with the features that have some variance. By default, the `VarianceThreshold` class will remove all features that have the same value throughout the dataset:

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

white_or_red_min_var = Pipeline([
    ('feature_selection', VarianceThreshold(threshold=0.01)), # keep features with variance > 0.01
    ('scale', StandardScaler()), 
    ('lr', LogisticRegression(random_state=0))
]).fit(w_X_train, w_y_train)

Check which features got removed:

In [ ]:
w_X_train.columns[
    ~white_or_red_min_var.named_steps[
        'feature_selection'
    ].get_support()
]

Performance doesn't change much when using just 9 out of the 11 features:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(
    w_y_test, white_or_red_min_var.predict(w_X_test)
))

### Principal Components Analysis
Can we see a way to easily separate these that might help us?

In [ ]:
from ml_utils.pca import pca_scatter
pca_scatter(wine_X, wine_y, 'wine is red?')
plt.title('Wine Kind PCA (2 components)')

In [ ]:
from ml_utils.pca import pca_scatter_3d
pca_scatter_3d(wine_X, wine_y, 'wine is red?', elev=20, azim=-10)
plt.suptitle('Wine Type PCA (3 components)')

How many PCA components explain most of the variance?

In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from ml_utils.pca import pca_explained_variance_plot

pipeline = Pipeline([
    ('normalize', MinMaxScaler()), ('pca', PCA(8, random_state=0))
]).fit(w_X_train, w_y_train) 

pca_explained_variance_plot(pipeline.named_steps['pca'])

Using a scree plot to find the number of components to use.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from ml_utils.pca import pca_scree_plot

pipeline = Pipeline([
    ('normalize', MinMaxScaler()), ('pca', PCA(8, random_state=0))
]).fit(w_X_train, w_y_train)

pca_scree_plot(pipeline.named_steps['pca'])

### Will a model fit on these components perform better?

In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('normalize', MinMaxScaler()),
    ('pca', PCA(4, random_state=0)),
    ('lr', LogisticRegression(
        class_weight='balanced', random_state=0
    ))
]).fit(w_X_train, w_y_train)

Notice we only have 4 features (our PCA components):

In [ ]:
pipeline.named_steps['lr'].coef_

Check the agreement between our new model and the original:

In [ ]:
# agreement with logistic regression alone
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(kind_preds, pipeline.predict(w_X_test))

Performance is still good using dimensionality reduction:

In [ ]:
from sklearn.metrics import classification_report

preds = pipeline.predict(w_X_test)
print(classification_report(w_y_test, preds))

In [ ]:
from ml_utils.classification import plot_roc
plot_roc(w_y_test, pipeline.predict_proba(w_X_test)[:,1])

In [ ]:
from ml_utils.classification import confusion_matrix_visual
confusion_matrix_visual(w_y_test, preds, ['low', 'high'])

## Can a decision tree tell us what features are important?
Decision trees give us feature importances. These sum up to 1 with the highest being more important.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=0).fit(w_X_train, w_y_train)
pd.DataFrame([(col, coef) for col, coef in zip(
    w_X_train.columns, dt.feature_importances_
)], columns=['feature', 'importance']
).set_index('feature').sort_values(
    'importance', ascending=False
).T

### Visualizing the top 2 features
Looking at the top 2 features, we can see if it is possible to separate red and white wine:

In [ ]:
sns.scatterplot(x=wine['total sulfur dioxide'], y=wine['chlorides'], hue=wine.kind, alpha=0.25)
plt.xscale('log')
plt.yscale('log')

### Using Top 2 Features for Classification of Red and White Wines
We can build a model with just these features:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

important_features = ['total sulfur dioxide', 'chlorides']
X_train = w_X_train[important_features]
X_test = w_X_test[important_features]

white_or_red_top_features = Pipeline([
    ('scale', StandardScaler()), 
    ('lr', LogisticRegression(random_state=0))
]).fit(X_train, w_y_train)

top_features_kind_preds = white_or_red_top_features.predict(X_test)

Notice our performance is still good:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(w_y_test, top_features_kind_preds))

In [ ]:
from ml_utils.classification import plot_roc

plot_roc(w_y_test, white_or_red_top_features.predict_proba(X_test)[:,1])

In [ ]:
from ml_utils.classification import confusion_matrix_visual

confusion_matrix_visual(w_y_test, top_features_kind_preds, ['white', 'red'])

### Visualizing the decision tree
We can also visualize the decisions the tree made:

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

graphviz.Source(export_graphviz(
    DecisionTreeClassifier(
        max_depth=2, random_state=0
    ).fit(w_X_train, w_y_train),
    feature_names=w_X_train.columns
))

Without graphviz:

In [ ]:
from sklearn.tree import plot_tree

plot_tree(
    DecisionTreeClassifier(
        max_depth=2, random_state=0
    ).fit(w_X_train, w_y_train),
    feature_names=w_X_train.columns
)

## Error Analysis on Logistic Regression
We can look at the incorrect predictions to get a better feel for our data and the model:

In [ ]:
prediction_probabilities = pd.DataFrame(
    white_or_red.predict_proba(w_X_test), 
    columns=['prob_white', 'prob_red']
).assign(
    is_red=w_y_test == 1,
    pred_white=lambda x: x.prob_white >= 0.5, 
    pred_red=lambda x: np.invert(x.pred_white),
    correct=lambda x: (np.invert(x.is_red) & x.pred_white)
                       | (x.is_red & x.pred_red)
)

prediction_probabilities.sample(5, random_state=0)

### Distribution of prediction confidence
When our model is confident, it is usually right:

In [ ]:
g = sns.displot(
    data=prediction_probabilities, x='prob_red',
    rug=True, kde=True, bins=20, col='correct', 
    facet_kws={'sharey': False}
)
g.set_axis_labels('probability wine is red', None)
plt.suptitle('Prediction Confidence', y=1.05)

### Are the incorrect classifications outliers?

In [ ]:
import math

incorrect = w_X_test.assign(is_red=w_y_test).iloc[prediction_probabilities.query('not correct').index]
chemical_properties = [col for col in wine.columns if col not in ['quality', 'kind']]
melted = wine.drop(columns='quality').melt(id_vars=['kind'])

fig, axes = plt.subplots(math.ceil(len(chemical_properties) / 4), 4, figsize=(15, 10))
axes = axes.flatten()

for prop, ax in zip(chemical_properties, axes):
    sns.boxplot(
        data=melted[melted.variable.isin([prop])], 
        x='variable', y='value', hue='kind', ax=ax,
        palette={'white': 'lightyellow', 'red': 'orchid'}, 
        saturation=0.5, fliersize=2
    ).set_xlabel('')
    for _, wrong in incorrect.iterrows():
        x_coord = -0.2 if not wrong['is_red'] else 0.2
        ax.scatter(x_coord, wrong[prop], marker='x', color='red', s=50)
    
# remove the extra subplots
for ax in axes[len(chemical_properties):]:
    ax.remove()
    
plt.suptitle(
    'Comparing Chemical Properties of Red and White Wines'
    '\n(classification errors are red x\'s)'
)
plt.tight_layout()

<hr>
<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
        <a href="../../ch_09/wine.ipynb">
            <button>&#8592; Chapter 9</button>
        </a>
        <a href="./planets_ml.ipynb">
            <button>Planets</button>
        </a>
        <a href="./red_wine.ipynb">
            <button>Red Wine</button>
        </a>
    </div>
    <div style="float: right;">
        <a href="../../solutions/ch_10/exercise_1.ipynb">
            <button>Solutions</button>
        </a>
        <a href="../ch_11/1-EDA_unlabeled_data.ipynb">
            <button>Chapter 11 &#8594;</button>
        </a>
    </div>
</div>
<hr>